In [1]:
import os
import torch
from botorch.test_functions.multi_fidelity import AugmentedHartmann

from botorch.models.gp_regression_fidelity import SingleTaskMultiFidelityGP
from botorch.models.transforms.outcome import Standardize
from gpytorch.mlls.exact_marginal_log_likelihood import ExactMarginalLogLikelihood

from botorch import fit_gpytorch_mll
from botorch.models.cost import AffineFidelityCostModel
from botorch.acquisition.cost_aware import InverseCostWeightedUtility
from botorch.acquisition import PosteriorMean
from botorch.acquisition.knowledge_gradient import qMultiFidelityKnowledgeGradient
from botorch.acquisition.fixed_feature import FixedFeatureAcquisitionFunction
from botorch.optim.optimize import optimize_acqf
from botorch.acquisition.utils import project_to_target_fidelity

from botorch.optim.optimize import optimize_acqf_mixed

from botorch.acquisition import qExpectedImprovement


tkwargs = {
    "dtype": torch.double,
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
}
SMOKE_TEST = os.environ.get("SMOKE_TEST")

In [13]:
problem = AugmentedHartmann(negate=True).to(**tkwargs)
fidelities = torch.tensor([0.5, 0.75, 1.0], **tkwargs)

bounds = torch.tensor([[0.0] * problem.dim, [1.0] * problem.dim], **tkwargs)
target_fidelities = {6: 1.0}

BATCH_SIZE=1

In [14]:
def generate_initial_data(n=16):
    # generate training data
    train_x = torch.rand(n, 6, **tkwargs)
    train_f = fidelities[torch.randint(3, (n, 1))]
    train_x_full = torch.cat((train_x, train_f), dim=1)
    train_obj = problem(train_x_full).unsqueeze(-1)  # add output dimension
    return train_x_full, train_obj

def initialize_model(train_x, train_obj):
    # define a surrogate model suited for a "training data"-like fidelity parameter
    # in dimension 6, as in [2]
    model = SingleTaskMultiFidelityGP(
        train_x, train_obj, outcome_transform=Standardize(m=1), data_fidelity=6
    )
    mll = ExactMarginalLogLikelihood(model.likelihood, model)
    return mll, model

In [15]:
cost_model = AffineFidelityCostModel(fidelity_weights={6: 1.0}, fixed_cost=5.0)
cost_aware_utility = InverseCostWeightedUtility(cost_model=cost_model)

In [16]:
def project(X):
    return project_to_target_fidelity(X=X, target_fidelities=target_fidelities)


def get_mfkg(model):

    curr_val_acqf = FixedFeatureAcquisitionFunction(
        acq_function=PosteriorMean(model),
        d=7,
        columns=[6],
        values=[1],
    )

    _, current_value = optimize_acqf(
        acq_function=curr_val_acqf,
        bounds=bounds[:, :-1],
        q=1,
        num_restarts=2,
        raw_samples=4,
        options={"batch_limit": 10, "maxiter": 200},
    )

    return qMultiFidelityKnowledgeGradient(
        model=model,
        num_fantasies=2,
        current_value=current_value,
        cost_aware_utility=cost_aware_utility,
        project=project,
    )

In [23]:
def optimize_mfkg_and_get_observation(mfkg_acqf):
    """Optimizes MFKG and returns a new candidate, observation, and cost."""

    print(type(mfkg_acqf))
    print(type(bounds))
    print(type(BATCH_SIZE))
    
    
    # generate new candidates
    candidates, _ = optimize_acqf_mixed(
        acq_function=mfkg_acqf,
        bounds=bounds,
        fixed_features_list=[{6: 0.5}, {6: 0.75}, {6: 1.0}],
        q=BATCH_SIZE,
        num_restarts=5,
        raw_samples=100,
        # batch_initial_conditions=X_init,
        options={"batch_limit": 5, "maxiter": 200},
    )

    # observe new values
    cost = cost_model(candidates).sum()
    new_x = candidates.detach()
    new_obj = problem(new_x).unsqueeze(-1)
    print(f"candidates:\n{new_x}\n")
    print(f"observations:\n{new_obj}\n\n")
    return new_x, new_obj, cost

In [26]:
train_x, train_obj = generate_initial_data(n=16)
print(train_x.shape, train_obj.shape)

torch.Size([16, 7]) torch.Size([16, 1])


In [25]:
cumulative_cost = 0.

for i in range(10):
    mll, model = initialize_model(train_x, train_obj)
    fit_gpytorch_mll(mll)
    mfkg_acqf = get_mfkg(model)
    new_x, new_obj, cost = optimize_mfkg_and_get_observation(mfkg_acqf)
    train_x = torch.cat([train_x, new_x])
    train_obj = torch.cat([train_obj, new_obj])
    cumulative_cost += cost
    

<class 'botorch.acquisition.knowledge_gradient.qMultiFidelityKnowledgeGradient'>
<class 'torch.Tensor'>
<class 'int'>
candidates:
tensor([[0.1828, 0.2752, 0.7674, 0.1858, 0.5187, 0.5716, 0.7500]],
       dtype=torch.float64)

observations:
tensor([[1.1540]], dtype=torch.float64)


<class 'botorch.acquisition.knowledge_gradient.qMultiFidelityKnowledgeGradient'>
<class 'torch.Tensor'>
<class 'int'>
candidates:
tensor([[0.2837, 0.1225, 0.8287, 0.1087, 0.5015, 0.5288, 0.5000]],
       dtype=torch.float64)

observations:
tensor([[0.8271]], dtype=torch.float64)


<class 'botorch.acquisition.knowledge_gradient.qMultiFidelityKnowledgeGradient'>
<class 'torch.Tensor'>
<class 'int'>
candidates:
tensor([[0.1712, 0.3058, 0.8222, 0.2070, 0.3268, 0.5527, 0.5000]],
       dtype=torch.float64)

observations:
tensor([[1.7560]], dtype=torch.float64)


<class 'botorch.acquisition.knowledge_gradient.qMultiFidelityKnowledgeGradient'>
<class 'torch.Tensor'>
<class 'int'>
candidates:
tensor([[0.1041, 0.3605,